# Source models for Nath & Thingbaijam (2012)

Read the source description input files from the online supplementary material and write them to XML.

In [ ]:
%matplotlib inline
%load_ext autoreload

In [ ]:
import os

%autoreload 2
import source_model_tools as smt

import numpy as np
import pandas as pd
import toolbox as tb
import lxml.etree as et
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from StringIO import StringIO
from IPython.display import display

import hmtk.sources as src
from openquake.hazardlib import tom, geo

from hmtk.plotting.mapping import HMTKBaseMap
from hmtk.parsers.source_model.nrml04_parser import nrmlSourceModelParser

In [ ]:
# define some lists needed at different stages
layer_ids = [1, 2, 3, 4]
layer_depths_km = [0, 25, 70, 180, 300]

min_mags = [4.5, 5.5]

# define the input file names from the original paper
model_path = 'nath2012probabilistic'
polygon_file_template = os.path.join(model_path,'polygonlay%d.txt')
seismicity_file_template = os.path.join(model_path,'seismicitylay%d.txt')
polygon_files = [polygon_file_template % i for i in layer_ids] 
seismicity_files = [seismicity_file_template % i for i in layer_ids] 

smoothed_data_template = os.path.join(model_path,'lay%dsmooth%.1f.txt')
smoothed_data_files = [[smoothed_data_template % (i, m) 
                        for i in layer_ids] for m in min_mags]

# implement some column-name replacements
variable_mapping = {'avalue': 'a', 'bvalue': 'b', 'stdbvalue': 'stdb'}

# an input file supplies some auxiliary data
aux_file = 'auxiliary data.csv'

# define prefixes for the output file names and models
areal_source_model_file = 'areal_source_model'
smoothed_source_data_file = 'smoothed_source_model'

In [ ]:
def read_polygons(file_name):
    """
    Read polygon descriptions from text file into pandas.DataFrame. 
    
    File format as per Nath & Thingbaijam (2012).
    """
    with open(file_name) as f:
        keys = f.readline().strip().split(',')
        result = []
        for line in f.readlines():
            values = line.strip().split(',', len(keys) - 1)
            entry = {}
            for key, value in zip(keys, values):
                key = key.strip('[]')
                value = value.strip('[]').replace(';', '\n')
                if key == 'zoneid':
                    value = int(value)
                else:
                    value = np.genfromtxt(StringIO(value), delimiter=',')
                entry[key] = value
            result.append(entry)
    return pd.DataFrame.from_dict(result)

In [ ]:
# read areal polygons and seismicity statistics for each layer
areal_df = pd.DataFrame()
areal_polygons_df = pd.DataFrame()
loop_data = zip(layer_ids, seismicity_files, polygon_files, 
                layer_depths_km[:-1], layer_depths_km[1:])
for layer_id, seismicity_file, polygon_file, z_min, z_max in loop_data:
        
    # read seismicity and polygons
    layer_seis_df = pd.read_csv(seismicity_file)
    layer_seis_df.rename(columns=variable_mapping, inplace=True)
    layer_poly_df = read_polygons(polygon_file)
    
    # fill in depths, specify source mechanisms, clean up dip & rake
    n_zones = len(layer_seis_df)
    idx = layer_seis_df.index
    layer_seis_df['zmin'] = pd.Series(np.full(n_zones, z_min), index=idx)
    layer_seis_df['zmax'] = pd.Series(np.full(n_zones, z_max), index=idx)
    layer_seis_df['layerid'] = pd.Series(np.full(n_zones, layer_id), index=idx)
    layer_seis_df['rake'] = tb.wrap(layer_seis_df['rake'])
    layer_seis_df['mechanism'] = pd.Series(
        smt.focal_mechanisms(layer_seis_df['dip'], layer_seis_df['rake']), index=idx)
    layer_seis_df.loc[layer_seis_df['dip'] == -1, 'dip'] = 45
    layer_seis_df.loc[layer_seis_df['mechanism'] == 'undefined', 'rake'] = 90
    layer_seis_df.loc[layer_seis_df['strike'] == -1, 'strike'] = 0
    layer_seis_df['mmin'] = pd.Series(np.full(n_zones, min_mags[0]), index=idx) 
    
    # put it all together
    layer_source_df = pd.merge(layer_seis_df, layer_poly_df, on='zoneid')
    areal_df = pd.concat([areal_df, layer_source_df], ignore_index=True)
    areal_polygons_df = pd.concat([areal_polygons_df, layer_poly_df], ignore_index=True)

# merge auxiliary data (crucially, tectonic zones)
aux_df = pd.read_csv(aux_file)
aux_df = aux_df.drop(['zmin','zmax','dip','rake','mechanism'], axis=1)
areal_df = pd.merge(areal_df, aux_df, on='zoneid')
areal_df['polygon coordinates'] = areal_df['polygon coordinates'].astype(np.ndarray)

# convert polygons coordinates to objects
areal_df['polygon'] = [
    geo.polygon.Polygon([geo.point.Point(lat, lon) 
                         for lat, lon in area_series['polygon coordinates']]) 
    for _, area_series in areal_df.iterrows()]

areal_df = smt.sort_and_reindex(areal_df)

In [ ]:
# show a summary including megathrust zones and bin statistics
drop_columns = ['tectonic zone', 'region', 'concerns', 'zmax', 'zmin',
                'polygon coordinates', 'polygon',
                'aspect ratio', 'dip', 'rake', 'strike']
display(pd.concat([areal_df.drop(drop_columns, axis=1).head(),
                   areal_df.drop(drop_columns, axis=1).tail()]))

In [ ]:
props = ['a','b','mmax']
ranges = [np.arange(-0.5, 8, 1), np.arange(-0.1, 1.7, 0.2), np.arange(-0.5, 10, 1)]
groups = areal_df.groupby('layerid')
fig, axes = plt.subplots(nrows=len(props), ncols=1, figsize=(6, 3*len(props)))
for prop, ax, bins in zip(props, axes, ranges):
    data = [group[prop] for _, group in groups]
    labels = ['layer %d' % id for id, _ in groups]
    ax.hist(data, label=labels, stacked=True, bins=bins)
    ax.set_ylabel(prop)
axes[0].legend(loc='upper left');
plt.savefig("ArealModelFmds.png", dpi=300,
            transparent=True, bbox_inches='tight', pad_inches=0.1)

In [ ]:
areal_df[areal_df['tectonic subregion'] == 'no seismicity'].drop(['polygon coordinates'], axis=1)

In [ ]:
# read smoothed seismicity data
smoothed_df_list = []
for mag, file_names in zip(min_mags, smoothed_data_files):
    
    layer_smoothed_df_list = []
    for layer_id, smoothed_file in zip(layer_ids, file_names):
        layer_smoothed_df = pd.read_csv(smoothed_file)
        nu_mag = 'nu%s' % str(mag).replace('.','_')

        renaming_dict = {nu_mag: 'nu', 'lat':'latitude', 'lon':'longitude'}
        layer_smoothed_df.rename(columns=renaming_dict, inplace=True)
        
        layer_smoothed_df['layerid'] = layer_id
        layer_smoothed_df['mmin model'] = mag
        layer_smoothed_df['mmin'] = mag

        layer_smoothed_df_list.append(layer_smoothed_df)
    
    layer_smoothed_df = pd.concat(layer_smoothed_df_list)
    smoothed_df_list.append(layer_smoothed_df)
    
smoothed_df = pd.concat(smoothed_df_list)
smoothed_df = smt.sort_and_reindex(smoothed_df)

display(pd.concat((smoothed_df.head(), smoothed_df.tail())))

In [ ]:
prop = 'nu'
fig, axes = plt.subplots(nrows=len(min_mags), ncols=1, figsize=(6, 3*len(min_mags)), sharex=True)
fig.subplots_adjust(hspace=0.1)
for min_mag, ax in zip(min_mags, axes):
    groups = smoothed_df[smoothed_df['mmin model'] == min_mag].groupby('layerid')
    data = [np.log10(group[prop]).values for _, group in groups]
    labels = ['layer %d' % id for id, _ in groups]
    ax.hist(data, label=labels, stacked=True, bins=np.arange(-4,1,0.5))
    ax.set_ylabel(('%s%g' % (prop, min_mag)).replace('.','_'))
axes[0].legend(loc='upper left')
plt.savefig("SmoothedModelActivityRates.png", dpi=300,
            transparent=True, bbox_inches='tight', pad_inches=0.1)

In [ ]:
# compute all distances
distances = np.full((len(smoothed_df), len(areal_df)), np.inf)
for i, area_series in areal_df.iterrows():
    at_depth = (smoothed_df['layerid'] == area_series['layerid']).values
    mesh = geo.mesh.Mesh(
        smoothed_df.loc[at_depth, 'longitude'].values,
        smoothed_df.loc[at_depth, 'latitude'].values)
    distances[at_depth, i] = area_series['polygon'].distances(mesh)

The truncated Gutenberg-Richter magnitude-frequency distribution in OpenQuake implements
$$\lambda(m \geq M) = 10^{a - b m} = e^{\alpha - \beta m}$$
where, since $\lambda$ is an annual rate, $10^a$ is too. If we ignore events below some threshold $m_{min}$ then the annual rate becomes
$$\lambda(m \geq m_{min}) = e^{\alpha - \beta m_{min}} e^{-\beta (m - m_{min})} = \nu e^{-\beta (m - m_{min})} $$
Thus to compute the $a$ value required by OpenQuake from the activity rate $\nu$ for a given magnitude threshold, we must also take into account the $b$ value for the zone:
$$a = \log_{10}(\nu) + b m_{min}$$


In [ ]:
this_model = smoothed_df['mmin model'] == 4.5
n_points_in_area = (distances == 0).sum(axis=0)
zone_id = 21
drop_cols = ['tectonic subregion','region', 'tectonic zone',
             'aspect ratio', 'polygon coordinates']
fmd_cols = ['zoneid','layerid','a','b','stdb','mmax','stdmmax']
zone_index = areal_df[areal_df['zoneid'] == zone_id].index
print n_points_in_area[zone_index][0]
display(areal_df[areal_df['zoneid'] == zone_id][fmd_cols])
print areal_df[areal_df['zoneid'] == zone_id]['b'].values[0]*5.5

In [ ]:
# for each point in the smoothed model, choose the closest areal zone 
# and copy some useful columns over
columns_to_copy = ['zoneid', 'zmax', 'zmin', 'tectonic subregion',
                   'a', 'b', 'stdb', 'mmax', 'stdmmax',
                   'rake', 'dip', 'strike', 'aspect ratio', 'msr']
index_min = distances.argmin(axis=1)
for i, area_series in areal_df.iterrows():
    picked = i == index_min
    for column in columns_to_copy:
        smoothed_df.loc[picked, column] = area_series[column]
    # grab mmax and bvalue from zone above if mmax zero for this zone
    if area_series['mmax'] == 0:
        alternate_zone = np.floor(area_series['zoneid']/10)
        index_alt = np.where(areal_df['zoneid'] == alternate_zone)[0]
        if len(index_alt) > 0:
            smoothed_df.loc[picked, 'mmax'] = areal_df.loc[index_alt[0], 'mmax']
            smoothed_df.loc[picked, 'b'] = areal_df.loc[index_alt[0], 'b']

# computing the a-value for each zone is now a cinch
smoothed_df['a zone'] = smoothed_df['a'].copy()
smoothed_df['a'] = smoothed_df['nu'] + smoothed_df['b']*smoothed_df['mmin model']

# for each area in the areal model: count the number of points and 
# sum the activity rates in the smoothed model. from the latter estimate
# an equivalent a-value
for mag in min_mags:
    this_model = smoothed_df['mmin model'] == mag
    sum_nu = np.array([smoothed_df.loc[(distance == 0) & this_model.values, 'nu'].sum()
          for distance in distances.T])
    areal_df['smoothed N ' +  str(mag)] = (distances[this_model.values, :] == 0).sum(axis=0)
    areal_df['smoothed nu ' +  str(mag)] = sum_nu.round(4)
    areal_df['smoothed a ' +  str(mag)] = (np.log10(nu_smoothed) + areal_df['b']*mag).round(2)
    areal_df['areal nu ' +  str(mag)] = (10**(areal_df['a'] - areal_df['b']*mag)).round(4)

In [ ]:
display_drop = ['zmax', 'zmin', 'aspect ratio', 'msr',
                'rake','dip','strike','stdb','stdmmax']
no_mmax_df = smoothed_df[smoothed_df['mmax'] == 0]
if len(no_mmax_df) > 0:
    print "Leftover points with no assigned mmax"
    display(no_mmax_df.drop(display_drop, axis=1).head())
no_b_df = smoothed_df[smoothed_df['b'] == 0]
if len(no_b_df) > 0:
    print "Leftover points with no assigned b"
    display(no_b_df.drop(display_drop, axis=1).head())
no_zoneid_df = smoothed_df[smoothed_df['zoneid'].isnull()]
if len(no_zoneid_df) > 0:
    print "Leftover points with no assigned zone id"
    display(no_zoneid_df.drop(display_drop, axis=1).head())

In [ ]:
pd.concat((smoothed_df.head(), smoothed_df.tail())).drop(display_drop, axis=1)

In [ ]:
this_lon_lat = (smoothed_df['longitude'] == 98) & (smoothed_df['latitude'] == 3.7)
display(smoothed_df[this_lon_lat].drop(display_drop, axis=1))

In [ ]:
areal_display_drop = display_drop + ['polygon coordinates','tectonic zone','concerns','layerid']
pd.concat((areal_df.head(), areal_df.tail())).drop(areal_display_drop, axis=1)

In [ ]:
nu_df = areal_df[['zoneid', 'layerid', 'areal nu 4.5', 'areal nu 5.5', 
                  'smoothed nu 4.5', 'smoothed nu 5.5', 'smoothed N 4.5', 'smoothed N 5.5']]
nu_df = nu_df.rename(columns={'areal nu 4.5': 'areal 4.5', 'smoothed nu 4.5': 'smoothed 4.5',
                              'areal nu 5.5': 'areal 5.5', 'smoothed nu 5.5': 'smoothed 5.5',
                              'smoothed N 4.5': 'N 4.5', 'smoothed N 5.5': 'N 5.5'})

for layer_id in layer_ids:
    series = pd.Series(nu_df[nu_df['layerid'] == layer_id].sum(axis=0))
    series['layerid'] = layer_id
    series['zoneid'] = 'All'
    nu_df = nu_df.append(series, ignore_index=True)

series = pd.Series(nu_df[nu_df['zoneid'] == 'All'].sum(axis=0))
series['layerid'] = 'All'
series['zoneid'] = 'All'
nu_df = nu_df.append(series, ignore_index=True)
nu_df['ratio 4.5'] = (nu_df['smoothed 4.5']/nu_df['areal 4.5']).round(2)
nu_df['ratio 5.5'] = (nu_df['smoothed 5.5']/nu_df['areal 5.5']).round(2)

random_zones = sorted(np.random.randint(len(areal_df), size=5))
pd.concat((nu_df.loc[random_zones, :], nu_df[nu_df['zoneid'] == 'All']))

In [ ]:
# try computing layers directly
nu_df = pd.DataFrame()
nu_df.index.name = 'layerid'

for layer_id in layer_ids:
    in_areal_layer = areal_df['layerid'] == layer_id
    in_smoothed_layer = smoothed_df['layerid'] == layer_id
    
    layer_series = pd.Series()
    for mag in min_mags:

        this_model = smoothed_df['mmin model'] == mag
        layer_series = layer_series.append(pd.Series({
            'areal ' + str(mag): (10**(areal_df[in_areal_layer]['a'] -  areal_df[in_areal_layer]['b']*mag)).sum().round(),
            'smoothed ' + str(mag): smoothed_df[in_smoothed_layer & this_model]['nu'].sum().round(), 
            }, name=layer_id))
    nu_df = nu_df.append(layer_series)

nu_df = nu_df.append(pd.Series(nu_df.sum(axis=0), name='Total'))
for mag in min_mags:
    nu_df['ratio ' + str(mag)] = (nu_df['smoothed ' + str(mag)]/nu_df['areal ' + str(mag)]).round()

display(nu_df)

In [ ]:
# compare to the catalogue
catalogue_df = pd.read_csv('../Catalogue/SACAT1900_2008v2.txt', sep='\t')
completeness_df = pd.read_csv('./thingbaijam2011seismogenic/Table1.csv', header=[0,1], index_col=[0,1])
completeness_df

In [ ]:
catalogue_df['SHOCK_TYPE'].value_counts().plot(kind='pie', figsize=(6, 6));

In [ ]:
# assign zones to events
catalogue_df['zoneid'] = -1
for i, area_series in areal_df.iterrows():
    at_depth = ((catalogue_df['DEPTH'] >= area_series['zmin']) & 
                (catalogue_df['DEPTH'] < area_series['zmax']))
    mesh = geo.mesh.Mesh(catalogue_df['LON'].values,
                         catalogue_df['LAT'].values)
    in_area = area_series['polygon'].distances(mesh) == 0
    catalogue_df.loc[at_depth & in_area, 'zoneid'] = area_series['zoneid']

In [ ]:
pd.concat((catalogue_df.head(), catalogue_df.tail()))

In [ ]:
# for each minimum magnitude and layer work out the activity rates
nu_cat_df = pd.DataFrame()
for layer_id, z_min, z_max in reversed(zip(layer_ids, layer_depths_km[:-1], layer_depths_km[1:])):
    layer_results = pd.Series()
    for mag in reversed(min_mags):
        above_thresh = catalogue_df['MAG_MW'] >= mag
        start = completeness_df[str(mag), 'start'][z_min, z_max]
        end = completeness_df[str(mag), 'end'][z_min, z_max]
        at_depth = ((catalogue_df['DEPTH'] >= z_min) & 
                    (catalogue_df['DEPTH'] < z_max))
        in_years = ((catalogue_df['YEAR'] >= start) & 
                    (catalogue_df['YEAR'] <= end))
        in_a_zone = catalogue_df['zoneid'] != -1
        is_mainshock = catalogue_df['SHOCK_TYPE'] == 'Mainshock'
        subcat_df = catalogue_df[above_thresh & at_depth & in_years & in_a_zone & is_mainshock]
        layer_results = layer_results.append(pd.Series({
                'cat ' + str(mag): len(subcat_df)/(end - start + 1),
            }, name=layer_id))
    nu_cat_df = nu_cat_df.append(layer_results)
nu_cat_df = nu_cat_df.append(pd.Series(nu_cat_df.sum(axis=0), name='Total'))
display(nu_df.join(nu_cat_df))

In [ ]:
pd.concat((subcat_df.head(), subcat_df.tail()))

In [ ]:
c = 85  # km correlation distance
delta = 0.1  # grid spacing
span = 2.5  # test grid span
lon = 80  # degrees
lat = 20  # degrees
point = geo.Point(lon, lat)
lon_list = np.arange(lon - span, lon + span + delta/2, delta)
lat_list = np.arange(lat - span, lat + span + delta/2, delta)
lon_limits = (lon - span - delta/2, lon + span + delta/2)
lat_limits = (lat - span - delta/2, lat + span + delta/2)
lon_mesh, lat_mesh = np.meshgrid(lon_list, lat_list)
mesh = geo.RectangularMesh(lon_mesh, lat_mesh)
distances = point.distance_to_mesh(mesh)
i_mid = int(span/delta)
span_km = (distances[i_mid,:].max(), distances[:,i_mid].max())
print('Grid extents %g and %g km' % span_km)
print('Must be at least 3x correlation distance or %g km' % (3*c))
if all(np.array(span_km) > 3*c):
    print 'OK!'
else:
    print 'Problem!'
weights = np.exp(-(distances/c)**2) * (distances < 3*c)
print(u'For correlation distance %g km and grid spacing %g°' % (c, delta))
print('the unnormalized weights sum to %g.' % weights.sum())

In [ ]:
plt.imshow(distances, origin='lower', aspect='equal', 
           interpolation='nearest', 
           extent=lon_limits + lat_limits)
plt.title('Test grid to compute smoothing normalization')
plt.xlabel(u'Longitude (°E)')
plt.ylabel(u'Latitude (°N)')
plt.grid()
plt.colorbar();

In [ ]:
plt.imshow(weights, cmap='hot', origin='lower', aspect='equal', 
           interpolation='nearest', 
           extent=lon_limits + lat_limits)
plt.title('Weights on test grid')
plt.xlabel(u'Longitude (°E)')
plt.ylabel(u'Latitude (°N)')
plt.grid()
plt.colorbar();

In [ ]:
min_mag = 5.5
layer_id = 1
model_layer = (smoothed_df['mmin model'] == min_mag) & \
    (smoothed_df['layerid'] == layer_id)
subset_df = smoothed_df[model_layer].copy()

lon_list = sorted(list(set(subset_df['longitude'])))
lat_list = sorted(list(set(subset_df['latitude'])))

lon_min, lon_max = [min(lon_list), max(lon_list)]
lat_min, lat_max = [min(lat_list), max(lat_list)]
lon_res = np.diff(lon_list).min().round(2)
lat_res = np.diff(lat_list).min().round(2)

lon_list = np.arange(lon_min, lon_max + lon_res, lon_res).round(2)
lat_list = np.arange(lat_min, lat_max + lat_res, lat_res).round(2)
lat_grid, lon_grid = np.meshgrid(lat_list, lon_list)

# assign known values
num_columns = ['a', 'b', 'nu']
txt_columns = ['tectonic subregion']
data = {}
for column in num_columns:
    data[column] = np.full_like(lon_grid, np.nan)
for column in txt_columns:
    data[column] = np.full_like(lon_grid, '', dtype='object')

for _, point_series in subset_df.iterrows():
    i = int(round((point_series['longitude'] - lon_min)/lon_res))
    j = int(round((point_series['latitude'] - lat_min)/lat_res))
    for column in num_columns:
        data[column][i, j] = point_series[column]
    for column in txt_columns:
        data[column][i, j] = point_series[column]

In [ ]:
param = 'a'
if param in ['nu']:
    log_scale = True
else:
    log_scale = False
limits = (np.nanmin(data[param]), np.nanmax(data[param]))
plt.hist(data[param].ravel(), range=limits, log=log_scale);
plt.savefig("SmoothedEquivalentMap_%s_mmin%g_layer%d.png" % (param, min_mag, layer_id), dpi=300,
            transparent=True, bbox_inches='tight', pad_inches=0.1)

In [ ]:
fig = plt.figure(figsize=(12,12))
ax = fig.gca()
if log_scale:
    plt.imshow(data[param], cmap='hot', origin='lower', aspect='equal', 
               extent=(lon_min, lon_max, lat_min, lat_max), 
               norm=LogNorm(vmin=limits[0], vmax=limits[1]))
else:
    plt.imshow(data[param], cmap='hot', origin='lower', aspect='equal', 
               extent=(lon_min, lon_max, lat_min, lat_max))

plt.colorbar(shrink=0.5)
ax.set_xlabel(u'Longitude (°E)')
ax.set_ylabel(u'Latitude (°N)')
ax.grid()
plt.savefig("SmoothedEquivalentMap_%s_mmin%g_layer%d.png" % (param, min_mag, layer_id), dpi=300,
            transparent=True, bbox_inches='tight', pad_inches=0.1)

In [ ]:
# write each layer of areal source model to KML with added binwise rates
areal_kml_df = smt.add_binwise_rates(areal_df)
for layer_id in layer_ids:
    this_layer = areal_kml_df['layerid'] == layer_id
    temp_df = areal_kml_df.drop(['layerid'], axis=1)
    smt.source_df_to_kml(temp_df.loc[this_layer, :], 
        '%s layer %d' % (areal_source_model_file, layer_id))

In [ ]:
# thin smoothed DataFrame so that resulting KML files are small enough to be read into QGIS
smoothed_kml_df = smoothed_df.copy()
drop_columns = ['zmax','zmin','aspect ratio','rake','dip','strike',
                'msr','stdb','stdmmax']
smoothed_kml_df.drop(drop_columns, axis=1, inplace=True)
res_deg = 0.2
this_lon_lat = ((((smoothed_kml_df['latitude'] % res_deg).round(2) % res_deg) == 0) &
                (((smoothed_kml_df['longitude'] % res_deg).round(2) % res_deg) == 0))
smoothed_kml_df = smoothed_kml_df[this_lon_lat]

# write each layer to KML with added binwise rates
for min_mag in reversed(min_mags):
    for layer_id in reversed(layer_ids):
        this_model_layer = ((smoothed_kml_df['mmin model'] == min_mag) &
                   (smoothed_kml_df['layerid'] == layer_id))
        temp_df = smoothed_kml_df.drop(['mmin model','layerid'], axis=1)
        smt.source_df_to_kml(temp_df.loc[this_model_layer, :], 
            '%s mmin %g layer %d' % (smoothed_source_data_file, min_mag, layer_id))

In [ ]:
# write areal model data to TSV file
areal_output_df = smt.add_binwise_rates(
    smt.twin_source_by_magnitude(areal_df))
areal_output_df.to_csv(areal_source_model_file + '.tsv', sep='\t')

In [ ]:
# write areal source model with megathrust sources twinned to NRML
areal_source_list = smt.source_df_to_list(
    smt.twin_source_by_magnitude(areal_df))
areal_source_model = src.source_model.mtkSourceModel(
    identifier='1', 
    name=model_path + ' areal', 
    sources=areal_source_list)
areal_source_model.serialise_to_nrml(areal_source_model_file + '.xml')

In [ ]:
# write point source models with megathrust sources twinned to NRML
for min_mag in min_mags:
    this_model = smoothed_df['mmin'] == min_mag 
    smoothed_source_list = smt.source_df_to_list(
        smt.twin_source_by_magnitude(smoothed_df.loc[this_model, :]))
    smoothed_source_model = src.source_model.mtkSourceModel(
        identifier='1', 
        name='%s smoothed m_min=%g' % (model_path, min_mag), 
        sources=smoothed_source_list)
    file_name = smoothed_source_data_file + '_mmin%g.xml' % min_mag 
    smoothed_source_model.serialise_to_nrml(file_name)

In [ ]:
class PseudoCatalogue:
    """
    ugly hack for plotting source mechanisms: 
    construct pseudo-cataloge from pandas.DataFrame
    """
    def __init__(self, source_model, select_depth='all'):
        data = []
        oq_sources = source_model.convert_to_oqhazardlib(tom.PoissonTOM(1.0))
        for source in oq_sources:
            
            longitude = np.mean(source.polygon.lons)
            latitude = np.mean(source.polygon.lats)
            strike = source.nodal_plane_distribution.data[0][1].strike
            dip = source.nodal_plane_distribution.data[0][1].dip
            rake = source.nodal_plane_distribution.data[0][1].rake
            magnitude = source.get_min_max_mag()[1]*2.5
            depth = source.hypocenter_distribution.data[0][1]
            name = source.id
            
            if select_depth == 'all' or depth == select_depth:
                data.append({'longitude': longitude, 'latitude': latitude,
                             'strike1': strike, 'dip1': dip, 'rake1': rake,
                             'magnitude': magnitude, 'depth': depth, 'id': name})
        self.data = pd.DataFrame(data)

    def get_number_tensors(self):
        return len(self.data.magnitude)

catalogue = PseudoCatalogue(areal_source_model)

In [ ]:
map_config = {"min_lon": 60, "max_lon": 105, 
              "min_lat": 0,  "max_lat": 40, "resolution": "l",
              "parallel_meridian_spacing": 5}
parser = nrmlSourceModelParser(areal_source_model_file + '.xml')

for depth in sorted(list(set(catalogue.data['depth']))):
    basemap = HMTKBaseMap(map_config, '')

    source_model_read = parser.read_file('Areal Source Model')
    selected_sources = [source for source in source_model_read.sources 
                    if source.hypo_depth_dist.data[0][1] == depth]
    source_model_read.sources = selected_sources    
    selected_catalogue = PseudoCatalogue(source_model_read)

    basemap.add_source_model(source_model_read, overlay=True) 
    basemap.add_focal_mechanism(selected_catalogue, magnitude=False)
    for _, item in selected_catalogue.data.iterrows():
        plt.annotate(s=item.id, xy=(item.longitude, item.latitude))
    plt.savefig("ArealModel%gkmDepth.png" % depth, dpi=300,
                transparent=True, bbox_inches='tight', pad_inches=0.1)